In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix

**Count matrix and metadata for VISp dataset**
 - Download the count data from Allen Institute portal
 - Convert to AnnData format - see [this getting started with AnnData tutorial](https://anndata-tutorials.readthedocs.io/en/latest/getting-started.html)
 - Save the resulting object for later use

In [2]:
# 1. Download count matrices from https://portal.brain-map.org/atlases-and-data/rnaseq/mouse-v1-and-alm-smart-seq or use the shell commands below.
# 2. Uncomment here to run these shell commands within the notebook. If in bash, remember to remove the leading "!"" 

# !curl -o VISp.zip https://celltypes.brain-map.org/api/v2/well_known_file_download/694413985
# !unzip -d VISp_dataset VISp.zip
# !rm VISp.zip

In [3]:
# Load VISp dataset
filename = './VISp_dataset/mouse_VISp_2018-06-14_exon-matrix.csv'
expr_df = pd.read_csv(filename, header=0, index_col=0, delimiter=',').transpose()
expr = expr_df.values

# Find gene names
filename = './VISp_dataset/mouse_VISp_2018-06-14_genes-rows.csv'
genes_df = pd.read_csv(filename, header=0, index_col=0, delimiter=',')
gene_symbol = genes_df.index.values
gene_ids = genes_df['gene_entrez_id'].values
gene_names = np.array([gene_symbol[np.where(gene_ids == name)[0][0]] for name in expr_df.columns])

# Get metadata and save restrict to relevant fields
filename = './VISp_dataset/mouse_VISp_2018-06-14_samples-columns.csv'
obs = pd.read_csv(filename, header=0, index_col=0, delimiter=',', encoding='iso-8859-1')

obs = obs.reset_index()
obs = obs[['sample_name','seq_name','class','subclass','cluster']]
obs = obs.rename(columns={'sample_name':'sample_id'})
obs = obs.set_index('sample_id')
obs.head()

,seq_name,class,subclass,cluster
sample_id,,,,
F1S4_160108_001_A01,LS-15006_S09_E1-50,GABAergic,Vip,Vip Arhgap36 Hmcn1
F1S4_160108_001_B01,LS-15006_S10_E1-50,GABAergic,Lamp5,Lamp5 Lsp1
F1S4_160108_001_C01,LS-15006_S11_E1-50,GABAergic,Lamp5,Lamp5 Lsp1
F1S4_160108_001_D01,LS-15006_S12_E1-50,GABAergic,Vip,Vip Crispld2 Htr2c
F1S4_160108_001_E01,LS-15006_S13_E1-50,GABAergic,Lamp5,Lamp5 Plch2 Dock5


In [4]:
# compose and store anndata object for efficient read/write
adata = ad.AnnData(X=csr_matrix(expr), dtype=expr.dtype)
adata.var_names = gene_names
adata.var.index.set_names('genes', inplace=True)
adata.obs = obs
adata.write('./VISp_dataset/VISp_dataset.h5ad')

**Filtering samples**

The next code block is optional, and requires `VISp_PROPOSE_metadata.csv` which contains:
- cell type labels at different resolutions of the taxonomy (see manuscript for details)
- sample ids to filter out non-neuronal cells

In the following, we will
1. restrict cells only to those samples specified in `VISp_PROPOSE_metadata.csv`
2. append metadata from `VISp_PROPOSE_metadata.csv` to the AnnData object
3. normalize counts, determine highly variable genes using scanpy functions
3. save a filtered AnnData object into a .h5ad file for subsequent use

In [5]:
adata = ad.read_h5ad('./VISp_dataset/VISp_dataset.h5ad')
propose_df = pd.read_csv('./VISp_dataset/VISp_PROPOSE_metadata.csv')
print(propose_df.head(2))
print(f'\nold shape: {adata.shape}')

adata = adata[adata.obs['seq_name'].isin(propose_df['seq_name']), :]
obs = adata.obs.copy().reset_index()
obs = obs.merge(right=propose_df, how='left', left_on='seq_name', right_on='seq_name')
obs = obs.set_index('sample_id')

adata = ad.AnnData(X=adata.X, dtype=adata.X.dtype, obs=obs, var=adata.var)
print(f'new shape: {adata.shape}')

             seq_name       cell_types_98 cell_types_50 cell_types_25
0  LS-15006_S09_E1-50  Vip Arhgap36 Hmcn1           n70           n66
1  LS-15006_S10_E1-50          Lamp5 Lsp1    Lamp5 Lsp1           n78

old shape: (15413, 45768)
new shape: (13349, 45768)


**Normalization and preliminary gene selection**

In [6]:
# transforms data in adata.X
adata.layers['log1pcpm'] = sc.pp.normalize_total(adata, target_sum=1e6, inplace=False)['X']

# transforms data in layers['lognorm'] inplace
sc.pp.log1p(adata, layer='log1pcpm')

# adds related columns to adata.var
sc.pp.highly_variable_genes(adata, layer='log1pcpm', flavor='cell_ranger',
                            n_top_genes=10000, inplace=True)


In [7]:
# restrict dataset to only highly variable genes:
adata_hvg = adata[:,adata.var['highly_variable']]
adata_hvg

View of AnnData object with n_obs × n_vars = 13349 × 10000
    obs: 'seq_name', 'class', 'subclass', 'cluster', 'cell_types_98', 'cell_types_50', 'cell_types_25'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    layers: 'log1pcpm'

In [8]:
# This is a sparse matrix
adata_hvg.layers['log1pcpm']

<13349x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 26077901 stored elements in Compressed Sparse Row format>

In [9]:
# For sparse matrix `M`, one can use `M.A` to convert it to dense
adata_hvg.layers['log1pcpm'].A[:5,:5]

array([[0.       , 3.8425822, 0.       , 0.       , 4.202729 ],
       [0.       , 4.164545 , 0.       , 0.       , 3.669378 ],
       [0.       , 3.8250968, 0.       , 0.       , 3.80287  ],
       [0.       , 3.935433 , 0.       , 0.       , 3.253237 ],
       [0.       , 5.4067717, 0.       , 0.       , 4.6442885]],
      dtype=float32)

**Parting notes:**

The anndata object created in this way has a few different fields that we will end up using with PROPOSE
1. The raw counts are in `adata.X`
2. The normalized counts (log1p of CPM values) are in `adata.layers['log1pcpm']`
3. All metadata (cell type labels etc. for supervised mode in PROPOSE) is in `adata.obs`
4. A coarse selection of genes is in `adata.var['highly_variable']`

In [10]:
# adata_hvg is a view. We'll convert it to an new AnnData object and write it out. 
adata_hvg = ad.AnnData(X=adata_hvg.X, dtype=adata_hvg.X.dtype,
                       obs=adata_hvg.obs, var=adata_hvg.var[['highly_variable']],
                       layers=adata_hvg.layers, uns=adata.uns)
adata_hvg.write('./VISp_dataset/VISp_filtered.h5ad')